<a href="https://colab.research.google.com/github/Osondu-ifunanya/Ecosystem-service-valuation-using-ML-and-spatial-metrics/blob/main/Ecosystem%20service%20valuation%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>